In [13]:
!pip install soynlp
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-0ecdh1zc
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-0ecdh1zc
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-9a76a_wj
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-9a76a_wj


In [18]:
import os
import pandas as pd
import re
from news_preprocessing import *

In [19]:
category = {'1': '경제', '2':'국제', '3':'문화', '4':'부동산', '5':'사회', '6':'생활건강', '7':'스포츠', '8':'연예', '9':'정치'}

In [20]:
path = '../data_kdx/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]
file_list_py

['MBN00003U_9_2017.csv',
 'MBN00003U_8_2017.csv',
 'MBN00003U_4_2017.csv',
 'MBN00003U_6_2016.csv',
 'MBN00003U_3_2016.csv',
 'MBN00003U_5_2017.csv',
 'MBN00003U_7_2016.csv',
 'MBN00003U_1_2018.csv',
 'MBN00003U_6_2017.csv',
 'MBN00003U_2_2018.csv',
 'MBN00003U_2_2016.csv',
 'MBN00003U_9_2016.csv',
 'MBN00003U_3_2018.csv',
 'MBN00003U_8_2016.csv',
 'MBN00003U_3_2017.csv',
 'MBN00003U_7_2018.csv',
 'MBN00003U_7_2017.csv',
 'MBN00003U_4_2016.csv',
 'MBN00003U_4_2018.csv',
 'MBN00003U_9_2018.csv',
 'MBN00003U_6_2018.csv',
 'MBN00003U_8_2018.csv',
 'MBN00003U_2_2017.csv',
 'MBN00003U_5_2016.csv',
 'MBN00003U_5_2018.csv']

In [21]:
file_dict_py = {}
for file in file_list_py:
    file_dict_py[file] = category[file[10]]
file_dict_py

{'MBN00003U_9_2017.csv': '정치',
 'MBN00003U_8_2017.csv': '연예',
 'MBN00003U_4_2017.csv': '부동산',
 'MBN00003U_6_2016.csv': '생활건강',
 'MBN00003U_3_2016.csv': '문화',
 'MBN00003U_5_2017.csv': '사회',
 'MBN00003U_7_2016.csv': '스포츠',
 'MBN00003U_1_2018.csv': '경제',
 'MBN00003U_6_2017.csv': '생활건강',
 'MBN00003U_2_2018.csv': '국제',
 'MBN00003U_2_2016.csv': '국제',
 'MBN00003U_9_2016.csv': '정치',
 'MBN00003U_3_2018.csv': '문화',
 'MBN00003U_8_2016.csv': '연예',
 'MBN00003U_3_2017.csv': '문화',
 'MBN00003U_7_2018.csv': '스포츠',
 'MBN00003U_7_2017.csv': '스포츠',
 'MBN00003U_4_2016.csv': '부동산',
 'MBN00003U_4_2018.csv': '부동산',
 'MBN00003U_9_2018.csv': '정치',
 'MBN00003U_6_2018.csv': '생활건강',
 'MBN00003U_8_2018.csv': '연예',
 'MBN00003U_2_2017.csv': '국제',
 'MBN00003U_5_2016.csv': '사회',
 'MBN00003U_5_2018.csv': '사회'}

다음과 같이 문장과 관련한 기능은 고려하지 않았다.

kss 문장분리  
remove_dup_sent       # 중복문장 제거하기  
spell_check_sent         # 맞춤법 보정  
morph_filter              # 문장 여부 확인  
excluded_word_filter    # 특정 단어를 포함하는 문장 필터링  
remove_stopwords      # 큰 의미가 없는 불용어 정의  
min_max_filter            # 문장을 최대, 최소 길이로 필터링합니다.  
spacing_sent             # 공백 보정하기  

remove_language        # 특정 언어 지우기  

In [23]:
df = pd.read_csv('../data_kdx/'+'MBN00003U_1_2018.csv', encoding='cp949')

In [25]:
df['MDA_ART_ESSN_NO']

0        3424432
1        3424434
2        3424435
3        3424448
4        3424511
          ...   
30579    3723955
30580    3723957
30581    3723959
30582    3723976
30583    3724027
Name: MDA_ART_ESSN_NO, Length: 30584, dtype: int64

In [31]:
mbn_news_wiki = {}
start_num = 0
for file_name, category in file_dict_py.items():
    df = pd.read_csv(path+file_name, encoding='cp949')
    df['category'] = category
    df['title'] = df['ART_SJ_CN']
    df['article'] = df['ART_CN']
    df['admin_id'] = "AIPaperboy"
    df['date'] = list(map(lambda x: str(x)[:4]+'-' + str(x)[4:6]+'-'+str(x)[6:], df['WRT_DATE']))
    df['news_id'] = df['MDA_ART_ESSN_NO']
    sample = df['ART_CN'] + df['ART_SJ_CN']
    sample = remove_html(sample)
    sample = remove_email(sample)
    sample = remove_hashtag(sample)
    sample = remove_user_mention(sample)
    sample = remove_url(sample)
    sample = remove_bad_char(sample)
    sample = remove_press(sample)
    sample = remove_copyright(sample)
    sample = remove_photo_info(sample)
    sample = remove_useless_breacket(sample)
    sample = remove_repeat_char(sample)
    sample = clean_punc(sample)
    sample = remove_repeated_spacing(sample)
    df['context'] = sample
    
    df['index_col'] = [str(i) for i in range(start_num, start_num+len(df), 1)]
    df = df.set_index(keys=['index_col'], inplace=False, drop=True)
    start_num = start_num+len(df)
    df_dict = df[['admin_id', 'news_id', 'category', 'date', 'title', 'article', 'context']].to_dict('index')
    mbn_news_wiki.update(df_dict)

In [32]:
len(mbn_news_wiki)

842904

In [33]:
mbn_news_wiki['0']

{'admin_id': 'AIPaperboy',
 'news_id': 3105313,
 'category': '정치',
 'date': '2017-01-01',
 'title': "'붉은 닭의 해' 정유년 첫 날, 대통령은?…靑관저서 참모들과 '떡국 조찬'",
 'article': '\'붉은 닭의 해\' 정유년 첫 날, 대통령은?…靑관저서 참모들과 \'떡국 조찬\'<br /><br /><br /><br /><!------------ PHOTO_POS_0 ------------><br /><br />박근혜 대통령이 새해 첫날인 1일 청와대 관저에서 참모들과 \'떡국 조찬\'을 합니다.<br /><br /><br /><br />2013년 취임 후 매년 1월 1일에는 국립현충원을 참배하고 국무위원 및 청와대 참모들과 떡국으로 아침을 먹었으나, 올해는 참배 없이 참모들과만 아침 식사를 같이하기로 했습니다.<br /><br /><br /><br />지난달 9일 국회의 탄핵소추안 가결로 직무가 정지된 상황임을 고려한 것입니다.<br /><br /><br /><br />박 대통령이 새해 첫날 내던 신년사와 국군 장병에 대한 격려 메시지도 올해는 황교안 대통령 권한대행 국무총리가 대신했습니다.<br /><br /><br /><br />이날 떡국 조찬 자리는 박 대통령이 최근 청와대 관저를 찾아온 참모들에게 "새해에 떡국이라도 드셔야 하지 않겠느냐"고 말한데 이어 참모들도 박 대통령에게 "혼자 드시지 마라"고 건의하면서 만들어진 것으로 알려졌습니다.<br /><br /><br /><br />새해 첫 식사자리인 만큼 건강 기원 등의 덕담이 서로 오가는 자리가 될 것으로 보입니다.<br /><br /><br /><br />다만 헌법재판소의 탄핵심판 절차가 본격화되고 박영수 특별검사팀의 수사도 속도를 내고 있어 이전처럼 편안한 분위기가 되기는 어려울 것으로 전망됩니다. <br /><br /><br /><br />특히 박 대통령의 퇴진을 촉구하는 여론이 계속되면서 박 대통령측 일각에

* 추가로 작업된 사항  
stopword : (앵커멘트, 질문 1, 질문 1 등 stopword에서 제외)  
bad_char : ☎      (▶는 하나의 정보가 될 수 있어 제거하지 않았다.)  
remove_press : [MBN 뉴스센터 / ], MBN뉴스 안병욱입니다.[최희석 기자]  
remove_copyright : [ & mk.co.kr, 무단전재 및 재배포 금지]  
remove_useless_breacket : 【】 , [ / ]            (적용 안됨 : 【 】[ ])  
(【 앵커멘트 】, 【 질문 1 】,【 질문2 】는 처리 안함), (영상취재 : 유용규, 영상편집 : 정민정 는 처리가 안됨)  

In [34]:
import json

with open("mbn_news_wiki.json", "w") as json_file:
    json.dump(mbn_news_wiki, json_file, indent=4, sort_keys=True)